In [1]:
from data_prepare import *
import os
import pandas as pd
# import warnings
# warnings.filterwarnings("ignore")

#### 提取并绘制标注区域

In [2]:
jf = r"D:\pad_before\ml\NG_0501\ng.json"
src = os.path.dirname(jf)
dst = src + "-defect"
offset = 30

extract_region(src, dst, jf, offset, square=True)

PAD_2B_0425_OK_outer_305-3_2_21_m0.jpg: 100% 1321/1321 [03:12<00:00,  6.87it/s]     

==提取并绘制标注区域完成==


#### 特征提取

In [3]:
jf = r"D:\pad_before\ml\NG_0501\ng.json"
orig_src = os.path.dirname(jf)
dst = os.path.dirname(jf)

# df, features = calcu_features(orig_src, jf, wanted_features=['mean', 'diff_dilate', 'diff_erode', 'var', 'skewness', 'kurtosis', 'maxVal', 'mean_filter'])
df, features = calcu_features(orig_src, jf)
df.to_excel(os.path.join(dst, 'features.xlsx'))
print(features)

calculate features: PAD_2B_0425_OK_outer_305-3_2_21_m0.jpg: 100% 1321/1321 [03:01<00:00,  7.30it/s]     


*********特征提取耗时 82.37112092971802
(2216, 27)
27
['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM', 'ostu_ret', 'mean', 'diff_dilate', 'diff_erode', 'var', 'skewness', 'kurtosis', 'area', 'perimeter', 'circularity', 'centripetal_moment_ratio', 'diagonal', 'angle', 'F1', 'F2', 'F3', 'F3-F1', 'FF', 'inertia_ratio', 'conDom', 'bg_mean']


#### 加类别

In [4]:
excel = r"D:\pad_before\ml\features.xlsx"
region_root = r"D:\pad_before\ml\NG_0501-defect"
add_labels(excel, region_root)

100%|██████████| 2216/2216 [00:06<00:00, 351.23it/s]


#### 保存数据

In [21]:
files = [i for i in os.listdir(r"\\Ds418\nas3\A-ZK\train_data\merge_1215\test-defect\1") if i.endswith("bmp")]
df.loc[files].to_excel(os.path.join(dst, 'features_1.xlsx'))

#### 聚类

In [24]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
from sklearn import preprocessing

df1 = df.loc[files]
X = np.array(df1)
# pca2d = PCA(n_components=2)
# X_std = preprocessing.scale(X) # 数据标准化
# X_2d = pca2d.fit_transform(X_std)# 数据降维至两维便于可视化

y_pred = KMeans(n_clusters=3, random_state=9).fit_predict(X)

In [26]:
import shutil

for index,i in enumerate(df1.index):
    y = y_pred[index]
    os.makedirs(os.path.join(r"C:\Users\1\Desktop\新建文件夹\4", str(y)), exist_ok=True)
    shutil.copy(os.path.join(r"C:\Users\1\Desktop\新建文件夹\4", i), os.path.join(r"C:\Users\1\Desktop\新建文件夹\4", str(y), i))

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn import preprocessing
import pandas as pd
import joblib
import os
from glob import glob
import shutil
from tqdm import tqdm


# 训练数据和测试数据的excel需要在不同的文件夹里
# 训练数据
excel_file = r"\\Ds418\nas\share\LX\tmp\OQC\station_cls_ext\2_2_ext - 副本\data.xlsx"
n_clusters = 20
df = pd.read_excel(excel_file, index_col='jpg')
X = np.array(df.drop(['label'], 1))
X_std = preprocessing.scale(X) # 数据标准化
km = KMeans(n_clusters=n_clusters, random_state=9).fit(X_std)
joblib.dump(km, os.path.join(os.path.split(excel_file)[0], 'km{}.pkl'.format(n_clusters)))
y_pred = km.predict(X_std)
df_km = pd.DataFrame({"real": df.label, "km": y_pred}, index=df.index)
df_km.to_excel(os.path.join(os.path.split(excel_file)[0], 'km{}_result.xlsx'.format(n_clusters)))

# df_km = pd.read_excel(os.path.join(os.path.split(excel_file)[0], 'km{}_result.xlsx'.format(n_clusters)), index_col='Unnamed: 0')

for i in tqdm(df_km.index):
    img_g = glob(os.path.join(r"\\Ds418\nas\share\LX\tmp\OQC\station_cls_ext\2_2_ext - 副本", "*", i))[0]
    dst = os.path.join(os.path.split(excel_file)[0], "data", "{}".format(df_km.loc[i, "km"]))
    # print(dst)
    os.makedirs(dst, exist_ok=True)
    shutil.copy(img_g, os.path.join(dst, i))


# # 测试数据
# excel_file = r"\\Ds418\nas3\A-ZK\train_data\merge_1215\test\features_with_labels_test_7.xlsx"
# model_file = r"\\Ds418\nas3\A-ZK\train_data\merge_1215\train\km.pkl"
# km = joblib.load(model_file)
# df = pd.read_excel(excel_file, index_col='Unnamed: 0')
# X = np.array(df.drop(['labels'], 1))
# y_pred = km.predict(X)
# df_km = pd.DataFrame({"real": df.labels, "km": y_pred}, index=df.index)
# df_km.to_excel(os.path.join(os.path.split(excel_file)[0], 'km_result.xlsx'))


# # 筛选指定类数据
# excel_file = r"\\Ds418\nas3\A-ZK\train_data\merge_1215\clus\clus_all\features_with_labels.xlsx"
# excel_file_km = r"\\Ds418\nas3\A-ZK\train_data\merge_1215\clus\clus_all\km20_result.xlsx"
# label = 6
# df = pd.read_excel(excel_file, index_col='Unnamed: 0')
# df_km = pd.read_excel(excel_file_km, index_col='Unnamed: 0')
# index_list = list(df_km[df_km["km"] == label].index)
# df_sub = df.loc[index_list,:]
# df_sub.to_excel(os.path.join(os.path.split(excel_file)[0], 'km_result_label{}.xlsx'.format(label)))

#### 加载数据

In [35]:
data = pd.read_excel(r"\\Ds418\nas3\A-ZK\train_data\merge\features_4huidi.xlsx")
data

,Unnamed: 0,mean,diff_dilate,diff_erode,var,skewness,kurtosis,maxVal,mean_filter
0,AZK1_20221122_PS_3-3_1_16_0034_17.bmp,51.149351,-6.787787,0.038768,68.406266,1.462110,3.481912,2.465985,69.096774
1,AZK1_20221122_PS_3-3_1_17_0035_0.bmp,71.175904,-20.300410,0.000000,512.438937,1.041585,0.450403,3.350642,83.665354
2,AZK1_20221122_YS_14-1_1_32_0014_1.bmp,52.797753,-9.139249,0.000000,31.487186,0.420621,-0.132582,1.218155,63.357143
3,AZK1_20221122_YS_14-1_1_32_0014_2.bmp,48.443235,-3.211980,0.000000,15.696233,-0.676603,1.292061,1.704954,0.000000
4,AZK1_20221122_YS_14-1_1_32_0014_3.bmp,50.554348,-5.474821,0.000000,22.044148,0.375177,-0.028274,2.465376,62.482759
...,...,...,...,...,...,...,...,...,...
90,AZK_1129_NG_0066_10-1_1_31_0.bmp,102.330144,-17.909583,0.000000,1446.728326,0.082768,-1.066088,2.910428,113.051724
91,AZK_1129_NG_0068_11-1_1_26_0.bmp,107.128322,-23.133552,0.741988,1221.097429,0.547897,-0.234075,3.917265,111.199836
92,AZK_1129_NG_0079_11-1_1_4_0.bmp,67.067961,-7.278487,0.691533,543.238100,3.868219,17.346514,3.925717,76.926606
93,AZK_1129_NG_0086_31-1_1_17_0.bmp,77.745434,-42.547249,-27.098443,4219.065333,1.609664,1.637837,8.052624,129.869452


#### 复制图像到指定类别

In [36]:
import shutil
for i in data[data["maxVal"]>4]["Unnamed: 0"]:
    shutil.move(os.path.join(r"C:\Users\1\Desktop\新建文件夹\4\huidi", i), os.path.join(r"C:\Users\1\Desktop\新建文件夹\4\huidi", "w_over_4", i))

In [42]:
for i in data[data["maxVal"]<=4][data["mean"]-data["mean_filter"]<=0][data["mean_filter"]>80]["Unnamed: 0"]:
    shutil.move(os.path.join(r"C:\Users\1\Desktop\新建文件夹\4\huidi\others", i), os.path.join(r"C:\Users\1\Desktop\新建文件夹\4\huidi\others", "m_over_80", i))